# Predicting heart disease using Machine Learning
This notebook looks into using various Python-based Machine Learning and Data Science
libraries in an attempt to build a machine learning model capable of predicting whether 
or not someone has heart-disease based on their medical attributes.

I am going to take the following approach:
1. Problem definition
2. Data
3. Evaluation
4. Features
5. Modeling
6. Experimentations

## 1. Problem Definition

In a statement,
> Given clinical parameters about a patient, can we predict whether or not they have
heart disease?

## 2. Data
The original data came from the Cleavland data from the UCI Machine Learning Repository.
https://archive.ics.uci.edu/dataset/45/heart+disease

There is also a version of it available on kaggle.
https://www.kaggle.com/c/heart-disease-uci/data

## 3. Evaluation
> If I can reach 95% accuracy at predicting whether or not a patient has heart disease
during the proof of concept, I will pursue the project.

## 4. Features

This is where you will get different information about the each of the features in the data.

**Create data dictionary**

* age. The age of the patient.
* sex. The gender of the patient. (1 = male, 0 = female).
* cp. Type of chest pain. (1 = typical angina, 2 = atypical angina, 3 = non — anginal pain, 4 = asymptotic).
* trestbps. Resting blood pressure in mmHg.
* chol. Serum Cholestero in mg/dl.
* fbs. Fasting Blood Sugar. (1 = fasting blood sugar is more than 120mg/dl, 0 = otherwise).
* restecg. Resting ElectroCardioGraphic results (0 = normal, 1 = ST-T wave abnormality, 2 = left ventricular hyperthrophy).
* thalach. Max heart rate achieved.
* exang. Exercise induced angina (1 = yes, 0 = no).
* oldpeak. ST depression induced by exercise relative to rest.
* slope. Peak exercise ST segment (1 = upsloping, 2 = flat, 3 = downsloping).
* ca. Number of major vessels (0–3) colored by flourosopy.
* thal. Thalassemia (3 = normal, 6 = fixed defect, 7 = reversible defect).
* num. Diagnosis of heart disease (0 = absence, 1, 2, 3, 4 = present).


# Preparing the tools

I am going to use Pandas,Numpy and Matplotlib for data analysis and Manipulation

In [1]:
# import all the tools I need

# Regular EDA and plotting libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# I want my plots to appear inside the notebook
%matplotlib inline

# Models from Scikit-learn
from sklearn.linear_model import LogisticRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Models Evaluations
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import RocCurveDisplay


C:\Users\Lenovo\Desktop\heart-disease-Project\env\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


ImportError: cannot import name 'LogisticRegressor' from 'sklearn.linear_model' (C:\Users\Lenovo\Desktop\heart-disease-Project\env\Lib\site-packages\sklearn\linear_model\__init__.py)